# [CDAF] Atividade 3

## Nome e matrícula
Nome: Luís Felipe Ramos Ferreira
Matrícula: 2019022553

## Referências
- [1] https://figshare.com/collections/Soccer_match_event_dataset/4415000
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.spadl.wyscout.convert_to_actions.html
- [3] https://github.com/TomDecroos/matplotsoccer
- [4] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PlottingShots.html
- [5] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PlottingPasses.html
- [6] https://soccermatics.readthedocs.io/en/latest/gallery/lesson1/plot_PassNetworks.html

## Questão 1
- Baixe o dataset 'Wyscout Europa Top 5 2017/2018' em [1].
- Escolha uma partida e carregue os dados de eventos em um dataframe do pandas.
- Converta os dados de eventos para SPADL [2].

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
BARCELONA_VS_REAL_MADRID_ID: int = 2565907
spain_df: pd.DataFrame = pd.read_json("../data/atv03/events/events_Spain.json")
match_df: pd.DataFrame = spain_df[spain_df["matchId"] == BARCELONA_VS_REAL_MADRID_ID]
match_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
593455,8,Simple pass,[{'id': 1801}],3321,"[{'y': 49, 'x': 50}, {'y': 40, 'x': 37}]",2565907,Pass,675,1H,3.274992,85,249644096
593456,8,Simple pass,[{'id': 1801}],14723,"[{'y': 40, 'x': 37}, {'y': 23, 'x': 30}]",2565907,Pass,675,1H,5.109197,85,249644097
593457,8,Simple pass,[{'id': 1801}],3306,"[{'y': 23, 'x': 30}, {'y': 57, 'x': 26}]",2565907,Pass,675,1H,7.109958,85,249644098
593458,8,Simple pass,[{'id': 1801}],3309,"[{'y': 57, 'x': 26}, {'y': 40, 'x': 8}]",2565907,Pass,675,1H,8.912297,85,249644099
593459,8,Simple pass,[{'id': 1801}],3915,"[{'y': 40, 'x': 8}, {'y': 14, 'x': 11}]",2565907,Pass,675,1H,11.290220,85,249644100
...,...,...,...,...,...,...,...,...,...,...,...,...
595243,3,Corner,"[{'id': 801}, {'id': 1801}]",4498,"[{'y': 100, 'x': 100}, {'y': 51, 'x': 88}]",2565907,Free Kick,675,2H,2982.539784,30,249646064
595244,1,Ground loose ball duel,"[{'id': 701}, {'id': 1802}]",3306,"[{'y': 51, 'x': 88}, {'y': 60, 'x': 88}]",2565907,Duel,675,2H,2983.141976,13,249646065
595245,1,Ground loose ball duel,"[{'id': 703}, {'id': 1801}]",3476,"[{'y': 49, 'x': 12}, {'y': 40, 'x': 12}]",2565907,Duel,676,2H,2983.907855,13,249646246
595246,2,Foul,[],3306,"[{'y': 60, 'x': 88}, {'y': 51, 'x': 88}]",2565907,Foul,675,2H,2984.949321,20,249646066


## Questão 2
- Visualize uma sequência de 5 ações da partida usando matplotsoccer.actions [3].

## Questão 3
- Visualize os chutes da partida, desenvolvendo seu código em cima do dataframe SPADL. Faça um plot para cada time. Adapte de [4].
- Qual time as melhores chances da partida? Por quê?

## Questão 4
- Escolha um jogador da partida que você escolheu.
- Faça um heatmap de todas ações dele [3].
- Faça um heatmap de todas as ações ofensivas dele [3].
- Faça um heatmap de todas as ações defensivas dele [3].
- O que você pode inferir sobre o comportamento do jogador? O comportamento dele varia muito do ataque para a defesa?

In [9]:
MESSI_ID: int = 3359
messi_df: pd.DataFrame = match_df[match_df["playerId"] == MESSI_ID]
messi_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
593485,8,Simple pass,[{'id': 1801}],3359,"[{'y': 87, 'x': 74}, {'y': 91, 'x': 80}]",2565907,Pass,676,1H,85.701277,85,249644574
593489,8,Simple pass,[{'id': 1801}],3359,"[{'y': 87, 'x': 67}, {'y': 71, 'x': 43}]",2565907,Pass,676,1H,92.114223,85,249644578
593499,8,Simple pass,[{'id': 1801}],3359,"[{'y': 66, 'x': 58}, {'y': 78, 'x': 75}]",2565907,Pass,676,1H,117.241765,85,249644587
593501,1,Ground attacking duel,"[{'id': 702}, {'id': 1801}]",3359,"[{'y': 67, 'x': 65}, {'y': 38, 'x': 63}]",2565907,Duel,676,1H,122.168204,11,249644589
593503,8,Simple pass,[{'id': 1801}],3359,"[{'y': 38, 'x': 63}, {'y': 43, 'x': 69}]",2565907,Pass,676,1H,124.686582,85,249644590
...,...,...,...,...,...,...,...,...,...,...,...,...
595055,10,Shot,"[{'id': 402}, {'id': 201}, {'id': 1206}, {'id'...",3359,"[{'y': 72, 'x': 96}, {'y': 0, 'x': 0}]",2565907,Shot,676,2H,2199.766741,100,249646166
595106,1,Ground attacking duel,"[{'id': 602}, {'id': 702}, {'id': 1801}]",3359,"[{'y': 83, 'x': 44}, {'y': 93, 'x': 36}]",2565907,Duel,676,2H,2411.569686,11,249646184
595116,7,Touch,[{'id': 1401}],3359,"[{'y': 43, 'x': 39}, {'y': 10, 'x': 35}]",2565907,Others on the ball,676,2H,2421.085231,72,249646190
595131,8,Launch,[{'id': 1802}],3359,"[{'y': 8, 'x': 44}, {'y': 42, 'x': 63}]",2565907,Pass,676,2H,2558.961654,84,249646196


## Questão 5
- Para o mesmo jogador, crie um mapa de passes com os passes que ele efetuou na partida, desenvolvendo seu código em cima do dataframe SPADL. Adapte de [5].
- O mapa de passes trouxe alguma informação nova sobre o jogador?

## Questão 6
- Crie uma rede de passes de cada uma das equipes, desenvolvendo seu código em cima do dataframe SPADL. Adapte de [6].
- O que você consegue inferir sobre a formação de cada equipe? Quais jogadores de cada equipe possuem o maior grau (tem maior soma do peso das arestas)?